In [1]:

import pandas as pd
import numpy as np
import folium
import geopandas
import geopy
from folium.plugins.marker_cluster import MarkerCluster
import folium.plugins as plugins
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [20]:
# Test
locator = Nominatim(user_agent='my_request')
location = locator.geocode('Bøgeparken 12')

print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

cleaned = pd.read_csv('cleaned_data.csv', sep=';')
cleaned


Latitude = 55.787265, Longitude = 12.515153


,Rum,Toilet,Boligareal,Grundareal,Kontantpris,Ejerudgift,Plan,latitude,longitude,city_distance,...,Garage_0,Garage_1,Drivhus_0,Drivhus_1,Erhverv_0,Erhverv_1,close_city_Aalborg,close_city_Aarhus,close_city_Copenhagen,close_city_Odense
0,6,2,190,1,4945000,3,1,55.76619,9.54160,60.51143,...,1,0,1,0,1,0,0,1,0,0
1,5,3,187,1,3995000,3,1,55.68278,9.56109,61.33861,...,1,0,1,0,1,0,0,0,0,1
2,4,2,146,930,3495000,2,2,55.69920,9.54617,63.09367,...,1,0,1,0,1,0,0,0,0,1
3,5,2,143,391,7395000,3,1,55.71265,9.53506,64.48529,...,1,0,1,0,1,0,0,0,0,1
4,4,2,150,617,2195000,2,2,55.70025,9.54549,63.19188,...,1,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4777,4,1,96,734,2875000,1,1,55.72645,12.35758,14.38579,...,1,0,1,0,1,0,0,0,1,0
4778,3,1,70,583,1395000,2,1,55.39549,10.42292,2.03041,...,1,0,1,0,1,0,0,0,0,1
4779,5,2,80,525,1499000,1,1,55.38837,10.41380,1.69283,...,1,0,1,0,1,0,0,0,0,1
4780,2,2,63,318,1175000,2,2,56.47082,10.02701,35.99106,...,1,0,1,0,1,0,0,1,0,0


In [27]:
cleaned_geo = cleaned.loc[:,['longitude','latitude','Kontantpris']]
cleaned_geo = cleaned_geo.dropna()
denmark_center = (56.2639, 9.5018)
"""
dk = folium.Map(
    location=denmark_center,
    tiles='cartodbpositron',
    zoom_start=7,
)
cleaned_geo.apply(lambda row:folium.CircleMarker(location=[row["latitude"],\
                                                           row["longitude"]],\
                                                           clustered_marker = True).add_to(map1), axis=1)
dk
"""
# Create a map
dk = folium.Map(
    location=denmark_center,
    tiles='cartodbpositron',
    zoom_start=7,
)

# Create a MarkerCluster layer
marker_cluster = MarkerCluster().add_to(dk)

# Iterate through the DataFrame rows and add markers to the MarkerCluster
for index, row in cleaned_geo.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        clustered_marker=True,
    ).add_to(marker_cluster)

dk.save('denmark_coordinates_map.html')

,longitude,latitude,Kontantpris
0,9.54160,55.76619,4945000
1,9.56109,55.68278,3995000
2,9.54617,55.69920,3495000
3,9.53506,55.71265,7395000
4,9.54549,55.70025,2195000
...,...,...,...
4777,12.35758,55.72645,2875000
4778,10.42292,55.39549,1395000
4779,10.41380,55.38837,1499000
4780,10.02701,56.47082,1175000


In [42]:
import folium
from folium.plugins import MarkerCluster

# Create a map
dk = folium.Map(
    location=denmark_center,
    tiles='cartodbpositron',
    zoom_start=7,
)


# Define a color function based on price values
def color_function(price):
    if price < 2000000:  # Adjust these thresholds as needed
        return 'green'
    elif price < 3500000:
        return 'orange'
    else:
        return 'red'


In [41]:

# Create a MarkerCluster layer
marker_cluster = MarkerCluster().add_to(dk)

# Iterate through the DataFrame rows and add markers with color based on prices
for index, row in cleaned_geo.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        clustered_marker=True,
        color=color_function(row["Kontantpris"]),  # Set color based on price
        fill=True,
        fill_opacity=0.4,
    ).add_to(marker_cluster)

folium.TileLayer('stamenterrain').add_to(dk)
# Display the map
dk.save('colored_markers_map.html')

In [40]:

folium.TileLayer('stamenterrain').add_to(dk)
# Iterate through the DataFrame rows and add markers with color based on prices
for index, row in cleaned_geo.iterrows():
    price_color = color_function(row["Kontantpris"])  # Get color based on price
    folium.CircleMarker(
        radius = 1,
        location=[row["latitude"], row["longitude"]],
        color=price_color,  # Set color based on price
        fill=True,
        fill_opacity=0.2,
    ).add_to(dk)

# Display the map
dk.save('colored_markers_map2.html')